In [37]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import warnings
from sklearn.metrics import roc_curve, auc

warnings.filterwarnings("ignore", category=RuntimeWarning)

ModuleNotFoundError: No module named 'matplotlib'

In [10]:
def maxtwoind_mammo(x):
    # x = x.to_numpy()  # แปลง DataFrame เป็น NumPy array
    y = []
    for i in range(x.shape[0]):
        n = np.argmax(x[i, :])  # ค้นหาตำแหน่งค่ามากที่สุด
        if n == 0:
            y.append([1, 0])
        elif n == 1:
            y.append([0, 1])
        else:
            print("error")
            break
    return np.array(y)

def maxtwoindclass_mammo(x):
    y = []
    for i in range(x.shape[0]):
        n = x[i, :]
        if np.array_equal(n, [1, 0]):
            x[i] = 1
        elif np.array_equal(n, [0, 1]):
            x[i] = 2
        else:
            print("Error")
            break
        y.append(x[i])
    return np.array(y)

def MultiClassMetrics_mammo(labels, predictions):
    confmat = confusion_matrix(labels, predictions)
    N = confmat.shape[0]

    Precision = np.zeros(N)
    Recall = np.zeros(N)
    Accuracy = np.zeros(N)
    TPR = np.zeros(N)
    FPR = np.zeros(N)

    if N > 2:
        for i in range(N):
            TP = confmat[i, i]
            FN = np.sum(confmat[i, :]) - TP
            FP = np.sum(confmat[:, i]) - TP
            TN = np.sum(confmat) - TP - FP - FN

            Precision[i] = (TP / (TP + FP)) * 100 if (TP + FP) > 0 else 0
            Recall[i] = (TP / (TP + FN)) * 100 if (TP + FN) > 0 else 0
            Accuracy[i] = ((TP + TN) * 100) / (TP + TN + FN + FP)
            TPR[i] = TP / (TP + FN) if (TP + FN) > 0 else 0
            FPR[i] = FP / (FP + TN) if (FP + TN) > 0 else 0

        # คำนวณค่าเฉลี่ย
        Precision = np.mean(Precision)
        Recall = np.mean(Recall)
        Accuracy = np.mean(Accuracy)
        TPR = np.mean(TPR)
        FPR = np.mean(FPR)

    else:  # กรณี Binary Classification
        TP = confmat[0, 0]
        FP = confmat[0, 1]
        FN = confmat[1, 0]
        TN = confmat[1, 1]

        Precision = (TP / (TP + FP)) * 100 if (TP + FP) > 0 else 0
        Recall = (TP / (TP + FN)) * 100 if (TP + FN) > 0 else 0
        Accuracy = ((TP + TN) * 100) / (TP + TN + FP + FN)
        TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
        FPR = FP / (FP + TN) if (FP + TN) > 0 else 0

    return Precision, Recall, Accuracy, TPR, FPR


def MultiClassMetrics_mammo_Train(predictions, labels):
    # Compute confusion matrix
    confmat = confusion_matrix(labels, predictions)
    N = confmat.shape[0]

    Accuracy = np.zeros(N)

    for i in range(N):
        TP = confmat[i, i]
        FN = np.sum(confmat[i, :]) - confmat[i, i]
        FP = np.sum(confmat[:, i]) - confmat[i, i]
        TN = np.sum(confmat) - TP - FP - FN

        Accuracy[i] = (TP + TN) / (TP + TN + FN + FP)

    # Compute average accuracy
    avg_accuracy = np.mean(Accuracy) * 100

    # Output
    return avg_accuracy


In [ ]:
X_train = pd.read_csv('./xTrain.csv',header=None)
t_train = pd.read_csv('./tTrain.csv',header=None)
b_train = pd.read_csv('./bTrain.csv',header=None)
X_test = pd.read_csv('./xTest.csv',header=None)
t_test = pd.read_csv('./tTest.csv',header=None)
b_test = pd.read_csv('./bTest.csv',header=None)

X_train = X_train.values
t_train = t_train.values
b_train = b_train.values
X_test = X_test.values
# t_test = t_test.values
b_test = b_test.values

print(X_train.shape)
print(t_train.shape)
print(b_train.shape)
print(X_test.shape)
# print(t_test.shape)
print(b_test.shape)

(1563, 14)
(1563, 2)
(1563, 1)
(395, 14)
(395, 1)


In [124]:
A1 = []
A2 = []
regpara = 1e-5
HiddenNode = 500
a1 = 0
a2 = 0
n = 1

In [125]:
import numpy as np

def Algo1(A, b, itrs, L_A, para_lam, theta, zigma, alpha):
    L = L_A / (np.linalg.norm(A) ** 2)
    m = A.shape[1]
    n = b.shape[1]
    x0 = np.zeros((m, n))
    x1 = x0 / 2
    x2 = x0 / 2
    
    para_lam = para_lam
    
    for i in range(itrs):
        theta = theta
        zigma = zigma
        alpha = alpha
        
        dn = x2 + theta * (x2 - x1) + zigma * (x1 - x0)
        
        calGrad = np.dot(A.T, np.dot(A, dn)) - np.dot(A.T, b)
        u = dn - L * calGrad
        c1 = np.maximum(np.abs(u) - para_lam * L, 0) * np.sign(u)
        calGrad1 = np.dot(A.T, np.dot(A, c1)) - np.dot(A.T, b)
        bn = c1 + L * (calGrad - calGrad1)

        IntU = bn - L * (np.dot(A.T, np.dot(A, bn)) - np.dot(A.T, b))
        Ub = np.maximum(np.abs(IntU) - para_lam * L, 0) * np.sign(IntU)
        y = alpha * dn + (1 - alpha) * Ub

        Inty = y - L * (np.dot(A.T, np.dot(A, y)) - np.dot(A.T, b))
        x = np.maximum(np.abs(Inty) - para_lam * L, 0) * np.sign(Inty)
        
        x0 = x1
        x1 = x2
        x2 = x
    
    return x


In [ ]:
inputWeight = pd.read_csv('./inputWeight.csv',header=None)
bias = pd.read_csv('./bias.csv',header=None)

bias=bias.values
inputWeight=inputWeight.values

In [127]:
import numpy as np
from time import time
i = 1
dim = X_train.shape[1]
A1 = []
A2 = []

a1, a2 = 0, 0

# while a1 < 80.0 or a2 < 90.0:
for i in range(11):
    H = 1 / (1 + np.exp(-(X_train @ inputWeight + np.tile(bias, (X_train.shape[0], 1)))))
    outputWeight = Algo1(H, t_train, n, L_A=0.999, para_lam=0.01, theta=0.999, zigma = -0.001, alpha = 0.8)
    outputTrain = np.dot(H, outputWeight)
    yTrain = maxtwoind_mammo(outputTrain) 

    predictionsTrain = maxtwoindclass_mammo(yTrain)
    predictionsTrain = predictionsTrain[:, 1]

    Traindata  = b_train
    precisionTrain , recallTrain , accuracyTrain , tpr_train, fpr_test = MultiClassMetrics_mammo(Traindata , predictionsTrain)

    # Test
    H_test = 1 / (1 + np.exp(-(X_test @ inputWeight + np.tile(bias, (X_test.shape[0], 1)))))
    outputTest = np.dot(H_test, outputWeight)

    yTest = maxtwoind_mammo(outputTest) 
    predictionsTest = maxtwoindclass_mammo(yTest)
    Testdata = b_test
    predictionsTest = predictionsTest[:, 1]
    precision, recall, accuracy, tpr, fpr = MultiClassMetrics_mammo(Testdata, predictionsTest)

    print(f"Iter: {n} Train Accuracy: {accuracyTrain:.2f} % Test Accuracy: {accuracy:.2f} % ")

    a1 = accuracyTrain
    a2 = accuracy
    A1.append(a1)
    A2.append(a2)

    n+=1

Iter: 1 Train Accuracy: 80.36 % Test Accuracy: 83.54 % 
Iter: 2 Train Accuracy: 80.49 % Test Accuracy: 84.05 % 
Iter: 3 Train Accuracy: 80.49 % Test Accuracy: 84.05 % 
Iter: 4 Train Accuracy: 80.49 % Test Accuracy: 84.30 % 
Iter: 5 Train Accuracy: 80.49 % Test Accuracy: 84.30 % 
Iter: 6 Train Accuracy: 80.49 % Test Accuracy: 84.30 % 
Iter: 7 Train Accuracy: 80.42 % Test Accuracy: 84.30 % 
Iter: 8 Train Accuracy: 80.49 % Test Accuracy: 84.30 % 
Iter: 9 Train Accuracy: 80.42 % Test Accuracy: 84.30 % 
Iter: 10 Train Accuracy: 80.49 % Test Accuracy: 84.30 % 
Iter: 11 Train Accuracy: 80.55 % Test Accuracy: 84.56 % 


# Inference

In [40]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

In [4]:
df = pd.read_excel('./Os2.xlsx').drop(columns=['Unnamed: 15'])
df

,Age,Gender,Hormonal Changes,Family History,Race/Ethnicity,Body Weight,Calcium Intake,Vitamin D Intake,Physical Activity,Smoking,Alcohol Consumption,Medical Conditions,Medications,Prior Fractures,Osteoporosis
0,69.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000
1,32.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,89.000000,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,3.000000,2.000000,2.000000,1.000000
3,78.000000,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000
4,38.000000,2.000000,2.000000,1.000000,3.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,39.101124,1.506639,1.498979,1.509704,2.025536,1.475485,1.512768,1.483657,1.478550,1.498468,1.495403,2.015832,1.496936,1.497957,0.500000
1959,90.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,1.000000
1960,18.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1961,21.355424,0.500084,0.500127,0.500034,0.818390,0.499526,0.499965,0.499860,0.499667,0.500125,0.500107,0.822682,0.500118,0.500124,0.500128


In [5]:
inputWeight = pd.read_csv('./inputWeightInference.csv',header=None).values
outputWeight = pd.read_csv('./outputWeightInference.csv',header=None).values
bias = pd.read_csv('./biasInference.csv',header=None).values

In [30]:
Test_DF = df.drop(columns=['Osteoporosis'])
Test_DF

,Age,Gender,Hormonal Changes,Family History,Race/Ethnicity,Body Weight,Calcium Intake,Vitamin D Intake,Physical Activity,Smoking,Alcohol Consumption,Medical Conditions,Medications,Prior Fractures
0,69.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,1.000000
1,32.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000
2,89.000000,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,3.000000,2.000000,2.000000
3,78.000000,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.000000
4,38.000000,2.000000,2.000000,1.000000,3.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,39.101124,1.506639,1.498979,1.509704,2.025536,1.475485,1.512768,1.483657,1.478550,1.498468,1.495403,2.015832,1.496936,1.497957
1959,90.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000
1960,18.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1961,21.355424,0.500084,0.500127,0.500034,0.818390,0.499526,0.499965,0.499860,0.499667,0.500125,0.500107,0.822682,0.500118,0.500124


In [33]:
df['Osteoporosis'].value_counts()

Osteoporosis
1.000000    980
0.000000    980
0.500000      1
0.500128      1
1.000255      1
Name: count, dtype: int64

In [41]:
X_new = Test_DF.iloc[1000].values
H_new = 1 / (1 + np.exp(-(X_new @ inputWeight + np.tile(bias, (X_new.shape[0], 1)))))
outputNew = np.dot(H_new, outputWeight)
conf = np.max(outputNew)
yNew = maxtwoind_mammo(outputNew)
predictionsNew = maxtwoindclass_mammo(yNew)
predictionsNew = predictionsNew[1, 1]
if predictionsNew == 1:
    print("Yes Osteoporosis")
elif predictionsNew == 0:
    print("No Osteoporosis")

print(f"Confidence: {conf*100 :.2f}  %")

Yes Osteoporosis
Confidence: 87.29  %
